In [114]:
import pandas as pd
import numpy as np

In [115]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [116]:
file_path = "/Users/afolabiadeshola/Downloads/dsp.csv"

EXPLORATION

In [117]:
with open(file_path, 'r') as f:
    names = f.readline()
cols = names.rstrip('\n').split(',')
drop_quotes = [column.replace('"', '') for column in cols]
print (len(drop_quotes))
print(drop_quotes)

69
['id', 'tap_id', 'useragent', 'ip_address', 'device_model', 'device_manufacturer', 'device_type', 'os_vendor', 'os_name', 'os_version', 'os_sub_version', 'browser_name', 'browser_version', 'location_continent', 'location_region', 'location_country', 'location_state', 'location_zip_code', 'user_id', 'ifa', 'dpifmd5', 'google_id', 'dsp_id', 'app_id', 'app_name', 'publisher_id', 'publisher_name', 'publisher_url', 'site_id', 'site_name', 'domain', 'referrer_url', 'site_link', 'data_type', 'exchange', 'carrier', '@source', 'captured_time', '@timestamp', 'source_type', 'device_screen_height', 'device_screen_width', 'device_screen_pixel_ratio', 'device_idfa', 'category', 'ad_id', 'win_price', 'campaign_id', 'campaign_name', 'campaign_type', 'campaign_level', 'campaign_status', 'campaign_flag', 'campaign_ad_type', 'cdn_flag', 'media_id', 'media_ad_url', 'media_type', 'media_level', 'media_status', 'media_flag', 'media_ad_id', 'media_ad_name', 'media_ad_width', 'media_ad_height', 'media_ad_f

There are 69 columns in total. There are alot of columns with NaN values. The idea is to exclude columns with at least 80% NaN values. The cells below also separate each of the columns based on its relevance.

In [118]:
totally_nan_columns =  ['os_sub_version','location_zip_code','publisher_name','publisher_url','domain','referrer_url','device_idfa','category','cdn_flag','media_id','media_ad_url','media_type','media_level','media_status','media_flag','media_ad_id','media_ad_name','media_ad_width','media_ad_height','media_ad_format','campaign_ad_domain','video_flag','campaign_landing_url']

In [119]:
needed_columns = [col for col in drop_quotes if col not in totally_nan_columns]

In [120]:
needed_columns

['id',
 'tap_id',
 'useragent',
 'ip_address',
 'device_model',
 'device_manufacturer',
 'device_type',
 'os_vendor',
 'os_name',
 'os_version',
 'browser_name',
 'browser_version',
 'location_continent',
 'location_region',
 'location_country',
 'location_state',
 'user_id',
 'ifa',
 'dpifmd5',
 'google_id',
 'dsp_id',
 'app_id',
 'app_name',
 'publisher_id',
 'site_id',
 'site_name',
 'site_link',
 'data_type',
 'exchange',
 'carrier',
 '@source',
 'captured_time',
 '@timestamp',
 'source_type',
 'device_screen_height',
 'device_screen_width',
 'device_screen_pixel_ratio',
 'ad_id',
 'win_price',
 'campaign_id',
 'campaign_name',
 'campaign_type',
 'campaign_level',
 'campaign_status',
 'campaign_flag',
 'campaign_ad_type']

In [121]:
IDs = ['id', 'tap_id', 'user_id', 'google_id','dsp_id', 'app_id', 'publisher_id', 'site_id', 'ad_id', 'campaign_id']

In [122]:
not_relevant = ['ip_address', 'useragent', '@source', 'source_type', 'location_continent', 'location_country', '@timestamp', 'win_price']

In [123]:
not_sure = ['dpifmd5', 'ifa', 'campaign_ad_type', 'campaign_name','campaign_type', 'campaign_level', 'campaign_status', 'campaign_flag', 'device_model', 'device_manufacturer']

In [127]:
target = ['data_type']

In [128]:
train_columns = [col for col in needed_columns if col not in list(np.concatenate((IDs, not_relevant, not_sure)))]

In [129]:
train_columns

['device_type',
 'os_vendor',
 'os_name',
 'os_version',
 'browser_name',
 'browser_version',
 'location_region',
 'location_state',
 'app_name',
 'site_name',
 'site_link',
 'data_type',
 'exchange',
 'carrier',
 'captured_time',
 'device_screen_height',
 'device_screen_width',
 'device_screen_pixel_ratio']

In [130]:
columns_with_nan = ['os_name', 'browser_name', 'location_region', 'location_state', 'app_name', 'site_name', 'site_link', 'carrier', 'inventory', 'os_vendor', 'os_version', 'browser_version', 'os_name_version', 'browser_name_version']

In [131]:
numerical_columms = ['device_screen_height', 'device_screen_width', 'device_screen_pixel_ratio']

In [132]:
top_attr = ['os_name_version', 'browser_name_version', 'inventory']

In [133]:
dictionary = {'Bid':0,'Click':1}

In [134]:
joined_columns = ['os_name', 'os_version', 'browser_name', 'browser_version', 'app_name', 'site_name', 'site_link', 'captured_time', 'data_type']

DATA EXPLORATION ON A SAMPLE SIZE OF 300K ROWS. WHATEVER INFERENCE GOTTEN FROM HERE WOULD BE USED TO GENERALIZE. TOTAL NUMBER OF ROWS IN THE DOCUMENT IS 1028856

In [135]:
df = pd.read_csv("/Users/afolabiadeshola/Downloads/dsp.csv", usecols=train_columns)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (29,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [136]:
data = df.copy()

HELPER FUNCTIONS

In [137]:
#The idea here is to add the app_name and site_name together to have a singular column inventory
def inventory(data,column1= 'app_name', column2 = 'site_name'):
    data['inventory'] = data[column1].replace(np.nan,'')+ data[column2].replace(np.nan,'')
    data['inventory'] = data['inventory'].replace('',np.nan)
#The split function gets the site_name from site link and helps reduce the Nan values 
def split(data,column1,column2):
    data[column2] = data[column1].str.split('/', expand = True)[2]
#A lot of columns were missing for region. The states column had less missing value. Since each state belong to a geographical region, it made sense to map them accordingly
def replace_region(data):
    region = data['location_state']
    if region in ['Benue','Kogi','Kwara','Nasarawa','Niger','Plateau','Abuja','Makurdi','Lokoja','Asokoro']:
        return 'North_Central'
    elif region in ['Adamawa','Bauchi','Borno','Gombe','Taraba','Yobe','Jos','Minna','Maiduguri','Yola']:
        return 'North_East'
    elif region in ['Jigawa','Kaduna','Kano','Katsina','Kebbi','Sokoto','Zamfara','Zaria','Dutse']:
        return 'North_West'
    elif region in ['Abia','Anambra','Ebonyi','Enugu','Imo','Abakaliki','Owerri','Umuahia']:
        return 'South_East'
    elif region in ['Akwa Ibom','Cross River','Bayelsa','Rivers','Delta','Edo','Benin City','Port Harcourt','Asaba',
                   'Warri','Nsukka','Calabar','Uyo','Yenagoa','Eket','Sagbama','Bonny','Effurun']:
        return 'South_South'
    elif region in ['Ekiti','Lagos','Ogun','Ondo','Osun','Oyo','Ikeja','Ikire','Badagri','Ikorodu','Ibadan',
                   'Suleja','Ilorin','Abeokuta','Osogbo','Akure','Ede','Ikotun','Lekki','Ikoyi','Ota','Ojota',
                   'Sagamu','Ogudu','Mowe','Agege','Omu-Aran','Aponri']:
        return 'South_West'
#This function takes care of missing values in the os_vendor column. Values from os_name can be used to fix this.
def replace_vendor(os_name,os_vendor):
    if 'Macintosh' in str(os_name):
        return 'Apple'
    elif 'X11' in str(os_name):
        return 'Microsoft'
    else:
        return os_vendor
#This cleans the time column and separates each value into day, hour and minute 
def clean_time(value):
    value = value.replace('T', ' ')
    value = value[:19]
    return value
def process_time(df):
    df['captured_time'] = df['captured_time'].apply(lambda x:clean_time(x))
    df['captured_time'] = pd.to_datetime(df['captured_time'])
    df['day'] = df['captured_time'].dt.day.astype('uint8')
    df['hour'] = df['captured_time'].dt.hour.astype('uint8')
    df['minute'] = df['captured_time'].dt.minute.astype('uint8')
#This functions clear Nans and replaces them with others for categorical variables and replaces numerical variables with the mean
def clear_nan(data):
    for item in columns_with_nan:
        data[item] = data[item].fillna("Others")
def clear_nan_numeric(data):
    for item in numerical_columms:
        data[item] = data[item].fillna(data[item].mean())
#This function picks top 50 values in a column and replaces every other value with others    
# def pick_top_attr(data):
#     for item in top_attr:
#         top_val = list(data[item].value_counts()[:50].index)
#         data[item] = data[item].apply(lambda x : x if x in top_val else "other")
#This is a general cleanup function. Converts columns to strings and joins columns together based on relevance.
def column_cleanup(data):
    data['y'] = data['data_type'].map(dictionary)
    data['os_version'] = data.os_version.astype(str)
    data['browser_version'] = data.browser_version.astype(str)
    data['location_region'] = data.apply(lambda x: replace_region(x), axis=1)
    data['os_vendor'] = data.apply(lambda x: replace_vendor(x['os_name'], x['os_vendor']), axis=1)
    data['os_name_version'] = data['os_name'] + data['os_version']
    data['browser_name_version'] = data['browser_name'] + data['browser_version']

def preprocessing_file(data):    
    process_time(data)
    split(data, 'site_link', 'site_name')
    inventory(data)
    column_cleanup(data)
    clear_nan_numeric(data)
    clear_nan(data) 
    #pick_top_attr(data)
    training_data = data.drop(joined_columns, axis=1)
    return training_data

In [138]:
final_data = preprocessing_file(data)

In [139]:
final_data.head()

,device_type,os_vendor,location_region,location_state,exchange,carrier,device_screen_height,device_screen_width,device_screen_pixel_ratio,day,hour,minute,inventory,y,os_name_version,browser_name_version
0,Phone,Google,South_West,Lagos,adx,/54,604.0,360.0,3.0,22,13,44,"Smart Scan ��� PDF Scanner, Free files Scanning",1,android6.0,Chrome Mobile WebView75.0
1,Phone,Google,South_West,Lagos,adx,/,602.0,360.0,3.0,22,14,47,"AppLock - Lock Apps, PIN u0026 Pattern Lock",1,android7.0,Chrome Mobile WebView58.0
2,Phone,Google,South_West,Lagos,adx,/,570.0,320.0,1.5,22,14,56,skpoints.com,1,android6.0,Chrome Mobile55.0
3,Phone,Google,South_West,Lagos,adx,/54,640.0,360.0,2.0,22,15,17,Emoji Launcher - Stickers u0026 Themes,1,android4.4.4,Chrome Mobile30.0
4,Phone,Google,South_West,Lagos,adx,/54,640.0,360.0,2.0,22,17,14,Proposal u0026 Surat Perjanjian,1,android7.0,Chrome Mobile WebView64.0


In [140]:
import yaml
category_dict = {}
def attribute(data,category_dict=None):
    config = yaml.safe_load(open("/Users/afolabiadeshola/Downloads/features.yml"))
    cat_attributes = config['cat_attributes']
    for attribute in cat_attributes:
        if attribute not in category_dict:
            category_dict[attribute] = set()
        category_dict[attribute].update(data[attribute].unique())

In [141]:
attribute(final_data, category_dict=category_dict)

In [142]:
categories = []
for category in category_dict.keys():
    categories.append(list(category_dict[category]))
    print(len(categories))

1
2
3
4
5
6
7
8
9


In [143]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

In [144]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [145]:
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])
cat_pipeline = Pipeline([
        ('one_hot_encoding', OneHotEncoder(handle_unknown = "ignore", categories = categories)),
    ])

In [146]:
import tensorflow.keras.backend as K
K.clear_session()

In [249]:
# def split_data(training_data):
    
#     num_attributes = training_data.select_dtypes(include=[np.number]).columns.tolist()
#     num_attributes.remove("y")

#     cat_attributes = training_data.select_dtypes(exclude=[np.number]).columns.tolist()

#     full_pipeline = ColumnTransformer([
#             ("num", num_pipeline, num_attributes),
#             ("cat", cat_pipeline, cat_attributes),
#         ])
    
#     X_raw = training_data.drop("y", axis=1)
#     y_r = training_data["y"].copy()
    
#     full_pipeline.fit(X_raw)
    
#     X_train_raw, X_cv_raw, y_train, y_cv = train_test_split(X_raw, y_r, test_size=0.2, stratify = y_r)
    
#     X_train = full_pipeline.transform(X_train_raw)
#     X_cv = full_pipeline.transform(X_cv_raw)
    
#     print(f"Clicks in Training Set =>  {np.sum(y_train == 1)} , in CV =>  {np.sum(y_cv == 1)}") 
#     print(f"SMS in Training Set =>  {np.sum(y_train == 0)} , in CV =>  {np.sum(y_cv == 0)}")
    
#     names = ["Extra Trees", "Random Forest", "KNeighbors","Logistic",
#          "Naive Bayes", "Decision Tree","Support Vector Machine"]
#     classifiers = [
#     ExtraTreesClassifier(n_estimators=200,criterion = 'entropy'),
#     RandomForestClassifier(n_estimators=200,criterion = 'entropy'),
#     KNeighborsClassifier(),
#     LogisticRegression(),
#     GaussianNB(),
#     DecisionTreeClassifier(criterion='entropy'),
#     SVC(kernel = 'rbf')
#     ]

#     i=0
#     f1_results=[]
#     acc_results=[]
#     for classifier in classifiers:
#         print(names[i])
#         classifier.fit(X_train, y_train)
#         y_pred = classifier.predict(X_cv)
#         f1score=f1_score(y_cv,y_pred)
#         accuracy=accuracy_score(y_cv,y_pred)
#         print("F1 Score:",f1score)
#         print("Accuracy Score:",accuracy)
#         f1_results.append(f1score)
#         acc_results.append(accuracy)
#         i+=1

In [147]:
from tensorflow.keras import layers

def get_wide_model():
    #inputs = tf.keras.Input(shape=X_train.shape[1:])
    inputs = tf.keras.Input(shape=(1906,))
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    deep_output = layers.Dense(1)(x)

    wide_output = layers.Dense(1)(inputs)
    total_output = layers.add([deep_output, wide_output])
    output = layers.Dense(1, activation= 'sigmoid')(total_output)
    
    model = tf.keras.Model(inputs, output, name='wide_and_deep') 
    
    
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=metrics)
    
    return model

In [148]:
import tensorflow as tf
metrics = [tf.keras.metrics.FalseNegatives(name='fn'),
           tf.keras.metrics.FalsePositives(name='fp'),
           tf.keras.metrics.TrueNegatives(name='tn'),
           tf.keras.metrics.TruePositives(name='tp'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall')]


In [149]:
wide_model = get_wide_model()

In [150]:
import os
import datetime
import tensorflow as tf
logdir = os.path.join("/Users/afolabiadeshola/Documents/DSP", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [151]:
wide_model.summary()

Model: "wide_and_deep"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1906)]       0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          244096      input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 128)          0           dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       dropout[0][0]                    
______________________________________________________________________________________

In [ ]:
# from imblearn.over_sampling import SMOTE
# smote_algo = SMOTE(random_state=0)
# smote_data_X,smote_data_Y = smote_algo.fit_sample(X_train, y_train)
# smote_data_X = pd.DataFrame(data=smote_data_X,columns=X_train.columns)
# smote_data_Y= pd.DataFrame(data=smote_data_Y,columns=["y"])

# X_train  = smote_data_X
# y_train  = smote_data_Y

In [152]:
def train_whole(df):
    
    num_attributes = df.select_dtypes(include=[np.number]).columns.tolist()
    num_attributes.remove("y")

    cat_attributes = df.select_dtypes(exclude=[np.number]).columns.tolist()

    full_pipeline = ColumnTransformer([
            ("num", num_pipeline, num_attributes),
            ("cat", cat_pipeline, cat_attributes),
        ])
    
    X_raw = df.drop("y", axis=1)
    y_r = df["y"].copy()
    
    full_pipeline.fit(X_raw)
    
    X_train_raw, X_cv_raw, y_train, y_cv = train_test_split(X_raw, y_r, test_size=0.2, stratify = y_r)
    
    X_train = full_pipeline.transform(X_train_raw)
    X_cv = full_pipeline.transform(X_cv_raw)
    
#     smote_algo = SMOTE(random_state=0)
#     smote_data_X,smote_data_Y = smote_algo.fit_sample(X_train, y_train)
#     smote_data_X = pd.DataFrame(data=smote_data_X,columns=X_train_raw.columns)
#     smote_data_Y= pd.DataFrame(data=smote_data_Y,columns=["y"])

#     X_train  = smote_data_X
#     y_train  = smote_data_Y
    
    
    print(f"Clicks in Training Set =>  {np.sum(y_train == 1)} , in CV =>  {np.sum(y_cv == 1)}") 
    print(f"SMS in Training Set =>  {np.sum(y_train == 0)} , in CV =>  {np.sum(y_cv == 0)}") 
    print (X_train.shape[1:])
    weight_for_0 = 1. / np.sum(y_train == 0)
    weight_for_1 = 1. / np.sum(y_train == 1)
    
    class_weight = {0: weight_for_0, 1: weight_for_1}
        
    wide_model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=2,
          validation_data=(X_cv, y_cv),
          callbacks = [tensorboard_callback],         
          class_weight=class_weight)
    score = wide_model.evaluate(X_cv, y_cv, verbose=0)
    predictions = wide_model.predict(X_cv)
    return wide_model
    print (predictions)        
    print('Validation Metrics    :', score)

In [153]:
train_whole(final_data)

Clicks in Training Set =>  1486 , in CV =>  372
SMS in Training Set =>  821598 , in CV =>  205399
(1906,)
Train on 823084 samples, validate on 205771 samples
Epoch 1/10
823084/823084 - 34s - loss: 1.1744e-06 - fn: 319.0000 - fp: 192548.0000 - tn: 629050.0000 - tp: 1167.0000 - precision: 0.0060 - recall: 0.7853 - val_loss: 0.4253 - val_fn: 43.0000 - val_fp: 41758.0000 - val_tn: 163641.0000 - val_tp: 329.0000 - val_precision: 0.0078 - val_recall: 0.8844
Epoch 2/10
823084/823084 - 54s - loss: 8.3857e-07 - fn: 175.0000 - fp: 129476.0000 - tn: 692122.0000 - tp: 1311.0000 - precision: 0.0100 - recall: 0.8822 - val_loss: 0.2881 - val_fn: 55.0000 - val_fp: 26012.0000 - val_tn: 179387.0000 - val_tp: 317.0000 - val_precision: 0.0120 - val_recall: 0.8522
Epoch 3/10
823084/823084 - 52s - loss: 7.6286e-07 - fn: 153.0000 - fp: 117007.0000 - tn: 704591.0000 - tp: 1333.0000 - precision: 0.0113 - recall: 0.8970 - val_loss: 0.3009 - val_fn: 53.0000 - val_fp: 25317.0000 - val_tn: 180082.0000 - val_tp: 31

In [177]:
model = train_whole(final_data)

Clicks in Training Set =>  1486 , in CV =>  372
SMS in Training Set =>  821598 , in CV =>  205399
(1906,)
Train on 823084 samples, validate on 205771 samples
Epoch 1/10
823084/823084 - 38s - loss: 6.3460e-07 - fn: 137.0000 - fp: 106864.0000 - tn: 714734.0000 - tp: 1349.0000 - precision: 0.0125 - recall: 0.9078 - val_loss: 0.2485 - val_fn: 37.0000 - val_fp: 23021.0000 - val_tn: 182378.0000 - val_tp: 335.0000 - val_precision: 0.0143 - val_recall: 0.9005
Epoch 2/10
823084/823084 - 54s - loss: 6.3031e-07 - fn: 138.0000 - fp: 105908.0000 - tn: 715690.0000 - tp: 1348.0000 - precision: 0.0126 - recall: 0.9071 - val_loss: 0.3275 - val_fn: 28.0000 - val_fp: 30443.0000 - val_tn: 174956.0000 - val_tp: 344.0000 - val_precision: 0.0112 - val_recall: 0.9247
Epoch 3/10
823084/823084 - 64s - loss: 6.0326e-07 - fn: 123.0000 - fp: 103300.0000 - tn: 718298.0000 - tp: 1363.0000 - precision: 0.0130 - recall: 0.9172 - val_loss: 0.2818 - val_fn: 37.0000 - val_fp: 23961.0000 - val_tn: 181438.0000 - val_tp: 33

In [240]:
model.save('model.h5')